In [1]:
#You need to install the Parameters.jl package: https://github.com/mauro3/Parameters.jl
#You need to install the Accessors.jl package: https://github.com/JuliaObjects/Accessors.jl
using Parameters  
using Accessors 
using LinearAlgebra
using Random 

@with_kw struct NetworkParameters #The @with_kw macro comes from the Parameters.jl package and makes nice constructors
    L::Int
    α_vector::Vector{Float64} #This vector is a vector of α_i which can then be scaled
    μ_vector::Vector{Float64} #This is the vector of service rates considered fixed
    P::Matrix{Float64} #routing matrix
    c_s::Float64 = 1.0 #The squared coefficient of variation of the service times with a default value of 1.0
end

############################
# Three queues in tandem
scenario1 = NetworkParameters(  L=3, 
                                α_vector = [0.5, 0, 0],
                                μ_vector = ones(3),
                                P = [0 1.0 0;
                                     0 0 1.0;
                                     0 0 0])

############################
# Three queues in tandem with option to return back to first queue
scenario2 = @set scenario1.P  = [0 1.0 0; #The @set macro is from Accessors.jl and allows to easily make a 
                                 0 0 1.0; # modified copied of an (immutable) struct
                                 0.3 0 0] 

############################
# A ring of 5 queues
scenario3 = NetworkParameters(  L=5, 
                                α_vector = ones(5),
                                μ_vector = collect(1:5),
                                P = [0  .8   0    0   0;
                                     0   0   .8   0   0;
                                     0   0   0    .8  0;
                                     0   0   0    0   .8;
                                     .8  0   0    0    0])

############################
# A large arbitrary network

#Generate some random(arbitrary) matrix P
Random.seed!(0)
L = 100
P = rand(L,L)
P = P ./ sum(P, dims=2) #normalize rows by the sum
P = P .* (0.2 .+ 0.7rand(L)) # multiply rows by factors in [0.2,0.9] 

scenario4 = NetworkParameters(  L=L, 
                                α_vector = ones(L),
                                μ_vector = 0.5 .+ rand(L),
                                P = P);

In [2]:
scenario3

NetworkParameters
  L: Int64 5
  α_vector: Array{Float64}((5,)) [1.0, 1.0, 1.0, 1.0, 1.0]
  μ_vector: Array{Float64}((5,)) [1.0, 2.0, 3.0, 4.0, 5.0]
  P: Array{Float64}((5, 5)) [0.0 0.8 … 0.0 0.0; 0.0 0.0 … 0.0 0.0; … ; 0.0 0.0 … 0.0 0.8; 0.8 0.0 … 0.0 0.0]
  c_s: Float64 1.0


In [3]:
"""
Compute the maximal value by which we can scale the α_vector and be stable.
"""
function maximal_alpha_scaling(net::NetworkParameters)
    λ_base = (I - net.P') \ net.α_vector #Solve the traffic equations
    ρ_base = λ_base ./ net.μ_vector #Determine the load ρ  
    return minimum(1 ./ ρ_base) #Return the maximal value by 
end

max_scalings = round.(maximal_alpha_scaling.([scenario1, scenario2, scenario3, scenario4]),digits=3)
println("The maximal scalings for scenarios 1 to 4 are: $max_scalings")

"""
Use this function to adjust the network parameters to the desired ρ⋆ and c_s
"""
function set_scenario(net::NetworkParameters, ρ::Float64, c_s::Float64 = 1.0)
    (ρ ≤ 0 || ρ ≥ 1) && error("ρ is out of range")  
    max_scaling = maximal_alpha_scaling(net)
    net = @set net.α_vector = net.α_vector*max_scaling*ρ
    net = @set net.c_s = c_s
    return net
end;


The maximal scalings for scenarios 1 to 4 are: [2.0, 1.4, 0.2, 0.216]


In [4]:
sc3_with_rho_09 = set_scenario(scenario3,0.9) #high load

NetworkParameters
  L: Int64 5
  α_vector: Array{Float64}((5,)) [0.1799999999999999, 0.1799999999999999, 0.1799999999999999, 0.1799999999999999, 0.1799999999999999]
  μ_vector: Array{Float64}((5,)) [1.0, 2.0, 3.0, 4.0, 5.0]
  P: Array{Float64}((5, 5)) [0.0 0.8 … 0.0 0.0; 0.0 0.0 … 0.0 0.0; … ; 0.0 0.0 … 0.0 0.8; 0.8 0.0 … 0.0 0.0]
  c_s: Float64 1.0


In [6]:
sc3_with_rho_02 = set_scenario(scenario3,0.2) #quite low load

NetworkParameters
  L: Int64 5
  α_vector: Array{Float64}((5,)) [0.03999999999999998, 0.03999999999999998, 0.03999999999999998, 0.03999999999999998, 0.03999999999999998]
  μ_vector: Array{Float64}((5,)) [1.0, 2.0, 3.0, 4.0, 5.0]
  P: Array{Float64}((5, 5)) [0.0 0.8 … 0.0 0.0; 0.0 0.0 … 0.0 0.0; … ; 0.0 0.0 … 0.0 0.8; 0.8 0.0 … 0.0 0.0]
  c_s: Float64 1.0


In [5]:
scenario3

NetworkParameters
  L: Int64 5
  α_vector: Array{Float64}((5,)) [1.0, 1.0, 1.0, 1.0, 1.0]
  μ_vector: Array{Float64}((5,)) [1.0, 2.0, 3.0, 4.0, 5.0]
  P: Array{Float64}((5, 5)) [0.0 0.8 … 0.0 0.0; 0.0 0.0 … 0.0 0.0; … ; 0.0 0.0 … 0.0 0.8; 0.8 0.0 … 0.0 0.0]
  c_s: Float64 1.0


In [7]:
function compute_ρ(net::NetworkParameters)
    λ = (I - net.P') \ net.α_vector #Solve the traffic equations
    return λ ./ net.μ_vector #This is the vector of ρ values
end

compute_ρ (generic function with 1 method)

In [8]:
compute_ρ(scenario3)

5-element Vector{Float64}:
 5.000000000000003
 2.5000000000000013
 1.6666666666666672
 1.2500000000000004
 1.0000000000000004

In [13]:
ρ_09 = compute_ρ(sc3_with_rho_09)

5-element Vector{Float64}:
 0.9
 0.45
 0.3
 0.22499999999999998
 0.18

In [14]:
ρ_02 = compute_ρ(sc3_with_rho_02)

5-element Vector{Float64}:
 0.2
 0.1
 0.06666666666666667
 0.05
 0.04

In [15]:
mean_qs_09 = ρ_09 ./ (1 .- ρ_09)

5-element Vector{Float64}:
 9.000000000000002
 0.8181818181818181
 0.4285714285714286
 0.29032258064516125
 0.2195121951219512

In [17]:
total_mean_q_09 = sum(mean_qs_09)

10.756588022520361

In [18]:
mean_qs_02 = ρ_02 ./ (1 .- ρ_02)

5-element Vector{Float64}:
 0.25
 0.11111111111111112
 0.07142857142857142
 0.052631578947368425
 0.04166666666666667

In [19]:
total_mean_q_02 = sum(mean_qs_02)

0.5268379281537177

# In the practical this week... 

It is like having...

In [20]:
P = [0 1 0 0;
     0 0 1 0;
     0 0 0 1;
     0 0 0 0]

4×4 Matrix{Int64}:
 0  1  0  0
 0  0  1  0
 0  0  0  1
 0  0  0  0